In [1]:
import numpy as np
import sys
import os
sys.path.insert(0, os.path.normpath(os.getcwd() + os.sep + os.pardir))
import utils

res = []
with open('../data/questions.json', 'r') as f:
        res = np.array([utils.remove_non_ascii(eval(data_point)['question']) for data_point in f.read().splitlines()])

In [2]:
res

array(['What does the collector do', 'Graph tests problems ',
       'Implementation of Garbage Collection - Mark and Sweep', ...,
       'Problems with A tests....help would be awesome ',
       'TrieMap broke.',
       "I don't really know how to randomly select a word from a TreeSet since it's not indexed. "],
      dtype='<U6494')

In [2]:
for question in res:
    print(question)
    print(utils.actual_question_preprocessor(question))
    print()

What does the collector do
what does the collector do

Graph tests problems 
graph tests problems

Implementation of Garbage Collection - Mark and Sweep
implementation of garbage collection - mark and sweep

I've problem with running time and complexity issues. My codes are otherwise fine I think
ive problem with running time and complexity issues my codes are otherwise fine i think

subtraction and multiplying, order of assembly operations.
subtraction and multiplying order of assembly operations

I'm having an issue with the Overflow test on my peek() method and I'm not sure how to fix it.
im having an issue with the errorcode test on my peek() method and im not sure how to fix it

questions about vector.c

questions about filename

debugging nearest neighbors
debugging nearest neighbors

We are having issues with removing stuff for the chaining hash dictionary
we are having issues with removing stuff for the chaining hash dictionary

evil random hang man chooser
evil random hang man

unsure how to get backing set (object isn't an IDictionary)
unsure how to get backing set object isnt an idictionary

Getting unexpected improper exceptions, not getting exceptions where I should be, in constructor and makeGuess
getting unexpected improper errorcode not getting errorcode where i should be in constructor and makeguess

char to byte
array queue
char to byte array queue

I would like to debug Graphs.. 
i would like to debug graphs

My tests disappeared...I updated my computer and something went wrong
my tests disappeared i updated my computer and something went wrong

trivial c question about structs
trivial c question about structs

To understand more about c tests
to understand more about c tests

Confused about initializing threads
confused about initializing threads

Cannot import class
cannot import class

unsure how to implement topksort

unsure how to implement topksort

unsure how to implement A Tests
unsure how to implement a tests

Freeing nodes and chunks at th

How do I add things to LinkedLists without looping
how do i add things to linkedlists without looping

In TrieMap, how do the iterators work for the generic type A? Also, how would you "concatenate" each result of iterator.next() onto an object in order to traverse down the trie?
in triemap how do the iterators work for the generic type a also how would you concatenate each result of iterator next() onto an object in order to traverse down the trie

Stack overflow error with dequeue.
stack errorcode error with dequeue

ArrayDeque class- issues with peekFront running constant time as well as importation errors with testing.
arraydeque class- issues with peekfront running constant time as well as importation errors with testing

Trying to make a helper method that returns a Map
trying to make a helper method that returns a map

Not sure why pointer

not sure why pointer

For projet 2, I tried to use some print statements and I'm not sure why the first time I try to print a pattern it see

i need help understanding the btests.
i need help understanding the btests

move to front dictionary put/get/remove
move to front dictionary put/get/remove

We are having issues with size and put/remove
we are having issues with size and put/remove

trouble with debugging part 7, not sure about typecasting for left shift
trouble with debugging part numericnumber not sure about typecasting for left shift

not sure why indexoutofbounds
not sure why indexoutofbounds

I'm confused on how to start the BeaverMap 
im confused on how to start the beavermap

Floating point error
floating nonnumericnumber error

I'm getting an error that the 
im getting an error that the

general debugging questions, code mostly works when using clang to run, but there is heap use after free error. Not sure whether to free or not to free. 

general debugging questions code mostly works when using clang to run but there is heap use after free error not sure whether to free or not to free

we are getting a null po

cant find why errorcode is occuring

We have an infinite loop in our part D program.
we have an infinite loop in our part d program

im bad
im bad

help with C tests 
help with c tests

On first clue, printing something but clearly isn't right. (doesn't mean anything, not nearly enough characters)
on first clue printing something but clearly isnt right doesnt mean anything not nearly enough characters

Accessing first index takes longer
accessing first index takes longer

I am not sure how EvilHangman works
i am not sure how evilhangman works

Memory addresses are not aligned
memory addresses are not aligned

Sorry I asked one of the TAs about this already but I'm still confused about randomSwap 
sorry i asked nonnumericnumber of the tas about this already but im still confused about randomswap

I'm trying to set up my atom with ssh and its not working. I've been sick the past few days so I haven't been able to do it.
im trying to set up my atom with ssh and its not working ive been si

...

im having trouble thinking through the implementation of keyset()

Running tests on computer
running tests on computer

Fixing issues with conditional
fixing issues with conditional

how to do part 1 of asm 
how to do part numericnumber of asm

not able to increment back pointer for B tests

not able to increment back pointer for b tests

I need help working on make guess to check the longest value
i need help working on make guess to check the longest value

Not sure why we are getting this one error
not sure why we are getting this nonnumericnumber error

ArrayDeque<> cannot 
arraydeque<> cannot

Buffer Overflow Exploits
buffer errorcode exploits

General help.


general help

Trying to debug LinkedDeque; says error is in peakFront but I can't see whats wrong
trying to debug linkeddeque says error is in peakfront but i cant see whats wrong

Enqueue is not working as intended
enqueue is not working as intended

C Test Section of Lab 3
c test section of lab numericnumber

question abou

iter Plot looks weird
iter plot looks weird

We are confused.
we are confused

I'm having trouble with resizing my ChainingHashDictionary. My code is also taking a while to run, which is a problem for the GitLab tests
im having trouble with resizing my chaininghashdictionary my code is also taking a while to run which is a problem for the gitlab tests

negative reclaimed number for garbage collector and make not working on gitlab but works locally
negative reclaimed number for garbage collector and make not working on gitlab but works locally

having trouble understanding number 5
having trouble understanding number numericnumber

Debugging epilogue infinite loop
debugging epilogue infinite loop

Lab code runs fine on command line, but returns an "Execution Error" without displaying outputs 
lab code runs fine on command line but returns an execution error without displaying outputs

Garbled bytes and overlapping payloads when implementing splitting
garbled bytes and overlapping payloa

JSON error in constructor for beaver graph
json error in constructor for beaver graph

I have memory leak errors, and I'm not sure where they are originating from.
i have memory leak errors and im not sure where they are originating from

I am having issues with testing if something is a Prefix for the c tests in trie map
i am having issues with testing if something is a prefix for the c tests in trie map

Test A remove issue
test a remove issue

Seems like ins-opt and initial are roughly the same? for append
seems like ins-opt and initial are roughly the same for append

Errrorrrrrrrrrrrrrrr
errrorrrrrrrrrrrrrrr

I am trying to figure out how the trie map works
i am trying to figure out how the trie map works

Ask if I've commented enough, if my pairwise summation explanation is correct, and how to not make my make clean function delete some stuff.
ask if ive commented enough if my pairwise summation explanation is correct and how to not make my make clean function delete some stuff



question about race conditions

I am trying to get the B tests to work with the TrieMap. So far, the C and D tests pass but the B tests seem to get stuck in a loop. I have tried to check if the term is a 'substring' of any of the keys in the TrieMap, however the test does not complete. I can successfully make the suffixes, however.
i am trying to get the b tests to work with the triemap so far the c and d tests pass but the b tests seem to get stuck in a loop i have tried to check if the term is a substring of any of the keys in the triemap however the test does not complete i can successfully make the suffixes however

AI Guesser is passing some test cases, but fails at a certain point. 
ai guesser is passing some test cases but fails at a certain nonnumericnumber

Meltdown is just running in an infinite loop
meltdown is just running in an infinite loop

trouble with page-load number in CLRU
trouble with page-load number in clru

regarding stage 2 ratios
regarding stage numericnumber 

value is

meltdown image started but only a black screen?
meltdown image started but only a black screen

Not passing assertion check. Tried adding if statements, adjusting variables.
not passing errorcode check tried adding if statements adjusting variables

Phase 5.                      


phase numericnumber

Debugging CircularArray enqueue 
debugging circulararray enqueue

I don't understand how the PACK function works in the textbook (i.e. unsure of how to encode data into the prologue/epilogue/header/footer)
i dont understand how the pack function works in the textbook i e unsure of how to encode data into the prologue/epilogue/header/footer

Can't get remove to work. I'm using a stack to track alphabets and nodes I've already traversed. Want to see if it works. Also, I don't think I understand the B tests conceptually at all.

cant get remove to work im using a stack to track alphabets and nodes ive already traversed want to see if it works also i dont think i understand the b t

In [815]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from word2number import w2n
from sklearn.metrics import f1_score, multilabel_confusion_matrix
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [710]:
import spacy
import en_core_web_sm
nlp = spacy.load('en')

In [759]:
tokenizer = RegexpTokenizer("[^;\s.?,!()]+\.c|[^;\s.,?!()]+\.py|[^;\s.?!(),]+\(\)|[^;\s.?,!()]+")
sentence = tokenizer.tokenize("Peter u better get ur ass over here".lower())
[WordNetLemmatizer().lemmatize(word) for word in sentence]

['peter', 'u', 'better', 'get', 'ur', 'as', 'over', 'here']

In [895]:
def load_data(filename, label_name):
    with open(filename, 'r') as f:
        return [[eval(data_point)['question'], eval(data_point)[label_name]] for data_point in f.read().splitlines() if label_name in eval(data_point)]

def is_TA_or_instructor_name(text):
    names = {"adam", "victor", "jesse", "peter"}
    
    if text in names:
        return True
    else:
        return False
    
def is_spelled_out_number(text):
    try:
        w2n.word_to_num(text)
        return True
    except ValueError:
        return False
    
def is_function(text): # change to regex
    if '()' in text:
        return True
    else:
        return False

def is_snake_case(text): # change to regex
    if '_' in text:
        return True
    else:
        return False

def is_filename(text):
    if '.c' in text or '.py' in text:
        return True
    else:
        return False
    
def preprocess_sentence(sentence):
    tokenizer = RegexpTokenizer("[^;\s.?,!()]+\.c|[^;\s.,?!()]+\.py|[^;\s.?!(),]+\(\)|[^;\s.?,!()]+")
    sentence = tokenizer.tokenize(sentence.lower())
    
    for i in range(len(sentence)):
        if sentence[i].isnumeric():
            sentence[i] = "numericnumber"
        elif is_spelled_out_number(sentence[i]):
            sentence[i] = "nonnumericnumber"
#         elif is_function(sentence[i]):
#             sentence[i] = "function"
        elif is_filename(sentence[i]):
            sentence[i] = "filename"
#         elif is_snake_case(sentence[i]):
#             sentence[i] = "snakecase"
        elif is_TA_or_instructor_name(sentence[i]):
            sentence[i] = "name"
        
    
    return ' '.join(sentence)

from sklearn.base import BaseEstimator, TransformerMixin

class questionLengthExtractor(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def transform(self, X, y=None):
        return [[len(sentence.split())] for sentence in X]

    def fit(self, X, y=None):
        return self 
    
def preprocess_data(x_data, vectorizer=None):
    sentence_lens = [[len(sentence.split())] for sentence in x_data]
#     processed_x_data = [preprocess_sentence(sentence) for sentence in x_data]
    processed_x_data = x_data
    X = None

    if vectorizer:
        X = vectorizer.transform(processed_x_data)
    else:
        vectorizer = CountVectorizer(preprocessor=preprocess_sentence)
#         vectorizer = TfidfVectorizer(preprocessor=preprocess_sentence)
        X = vectorizer.fit_transform(processed_x_data)
        
    X = np.hstack((X.toarray(), sentence_lens))
    
    return X, vectorizer

def prepare_y_binary(y_data, true):
    return [1 if label == true else 0 for label in y_data]


In [854]:
# tokenizer = RegexpTokenizer("[^;\s.?,!()]+\.c|[^;\s.,?!()]+\.py|[^;\s.?!(),]+\(\)|[^;\s.?,!()]+")

preprocess_sentence("I don't know what's wrong...")
from sklearn.naive_bayes import MultinomialNB

In [904]:
# data = np.array(load_data('../data/questions.json', 'question_type'))
data = np.array(load_data('../data/questions.json', 'actual_question'))
train_data, test_data = train_test_split(data)

In [905]:
X, vectorizer = preprocess_data(train_data[:,0])
clf = RandomForestClassifier()
# clf = MultinomialNB()
y = train_data[:,1]
# y = prepare_y_binary(train_data[:,1], 't')
clf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [906]:
X_test, _ = preprocess_data(test_data[:,0], vectorizer)
y_preds = clf.predict(X_test)
y_test = test_data[:,1]
# y_test = prepare_y_binary(test_data[:,1], 't')
cross_val_score(clf, X_test, y_test).mean(), sum(clf.predict(X) == y) / len(y), sum(y_preds == y_test) / len(y_preds)

(0.9818181818181818, 1.0, 0.9433962264150944)

In [907]:
for i in range(len(y_preds)):
    if y_preds[i] != y_test[i]:
        print("Question: {}, Prediction: , Actual: {}".format(test_data[:,0][i], clf.predict_proba([X_test[i]])[0][1], y_test[i]))

Question: Hello my name is , Prediction: , Actual: 0.64
Question: StackOverflowError in DFS, Prediction: , Actual: 0.5
Question: ooops we have a problem , Prediction: , Actual: 0.6


array(['f', 't'], dtype='<U597')

In [121]:
stop_words = stopwords.words('english')

In [133]:
for word in "I couldnt solve the FileNotFound Error?".lower().split():
    if word not in stop_words:
        print(word)

couldnt
solve
filenotfound
error?


In [144]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each